In [3]:
import os

import json as js
from openai import OpenAI


In [2]:
api_key = None
with open("../../.api-key", "r") as f:
    api_key = f.read().splitlines()[0]
    
## TODO: remove this line to get the original api-key ##
api_key+="abc"
#################### END TODO ##########################

client = OpenAI(api_key = api_key)

**Setup assistant**

In [ ]:
description = """"""
instructions = """"""
name = "Scientific-Research-Idea-Generator"
tools = [{"type": ["code_interpreter", "retrieval", "function"]}]
model = "gpt-4"

In [ ]:
my_assistant = client.beta.assistants.create(
    description=description,
    instructions=instructions,
    name=name,
    tools=tools,
    model=model
)

**Append the new assistant instance to local assistant list**

In [28]:
my_assistant_id = my_assistant.get('id')

file_path = "./data/assistant_list.json"
try:
    with open(file_path, 'r') as file:
        list_file = js.load(file)
    
    if not list_file:
        # the list is empty
        list_file = {"data": []}
        list_file['data'].append(my_assistant)
    else:
        for i, item in enumerate(list_file['data']):
            if item['id'] == my_assistant_id:
                list_file['data'][i] = my_assistant
                break
    with open(file_path, 'w') as file:
        js.dump(list_file, file, indent=4)  

except FileNotFoundError:
    print(f"File not found: {file_path}")
except js.JSONDecodeError:
    print("Invalid JSON file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

**Upload Files from ./data/assistant_files/\* to the OpenAI server**

In [ ]:
assistant_file_path = "./data/assistant_files/"
file_count = len([name for name in os.listdir(assistant_file_path) if os.path.isfile(os.path.join(assistant_file_path, name))])
print(f"Number of files: {file_count}")

for file_name in os.listdir(assistant_file_path):
    file_path = os.path.join(assistant_file_path, file_name)
    
    if os.path.isfile(file_path):
        client.files.create(
            file=open(file_path, "rb"),
            purpose="assistants"
        )

In [20]:
# Function to retrieve all files for "assistant"
def get_assitant_files(json_data):
    retrieved_files = []
    for item in json_data.get('data', []):
        if item.get('purpose') == "assistants":
            retrieved_files.append(item.get("id"))
    
    return retrieved_files

list_files = client.files.list()
list_files = js.loads(list_files)

if len(list_files["data"]) == 0:
    print("No file was uploaded")
else:
    retrieved_files = get_assitant_files(list_files)

print(f"Retrieved {len(retrieved_files)} of files")

Retrieved 2 of files


**Bind assistant with assistant files**

In [ ]:
original_number_of_files = len(my_assistant.get('file_ids'))

for file_id in retrieved_files:
  assistant_file = client.beta.assistants.files.create(
    assistant_id=my_assistant_id,
    file_id=file_id
  )

my_assistant = client.beta.assistants.retrieve(my_assistant_id)
new_number_of_files = len(my_assistant.get('file_ids'))
print(f"{new_number_of_files-original_number_of_files} of files added.")